# 스레드

스레드를 사용하면, 하나의 프로세스에 여러개의 흐름을 만들어서 병렬적으로 실행할 수 있게 한다.

1. CPU 사용률 향상
2. 효율적인 자원 활용 및 응답성 향상
3. 코드 간결 및 유지보수성 향상

하나의 process는 여러개의 thread를 가질 수 있다.

# 스레드를 만드는 방법

1. 함수 전달

2. 상속

### thread 객체에 함수를 전달해서 만드는 방법

```python
import threading

threading.Thread(name=, target=, args=)

```




```name : 스레드의 이름을 정한다. 주지 않아도 된다.
target: 스레드에서 실행할 함수
args: target 함수에 넘겨질 매개변수, 튜플형태로 전달한다.
```

### 스레드 관련 속성, 메서드

```start(): 스레드 실행
join(): 해당 스레드에서 실행되는 함수가 종료될 때까지 기다린다.
```

In [10]:
import threading
from time import sleep, ctime

In [11]:
# thread 객체에 함수를 전달해서 만드는 방법.

def hands():
    sleep(3)
    print(ctime())
    return print("손을 흔든다.")

def legs():
    sleep(2)
    print(ctime())
    return print("발을 흔든다.")

def head():
    sleep(1)
    print(ctime())
    return print("머리를 휘젓는다.")

In [12]:
# 싱글 스레드로 함수 호출하기

print("싱글 스레드 시작")

hands()
legs()
head()

print("싱글 스레드 종료")

싱글 스레드 시작
Sat Aug 29 23:15:23 2020
손을 흔든다.
Sat Aug 29 23:15:25 2020
발을 흔든다.
Sat Aug 29 23:15:26 2020
머리를 휘젓는다.
싱글 스레드 종료


In [13]:
t1 = threading.Thread(target=hands)
t2 = threading.Thread(target=legs)
t3 = threading.Thread(target=head)

t1.start()


t2.start()

t3.start()


스레드객체.join()은 해당 스레드가 종료될 때 까지 다른 스레드를 실행하지 않겠다는 의미이다.

```pyhton

# 1번
for t in threads:
    t.start()
    t.join
    
    
# 2번
for t in threads:
    t.start()
    
for t in threads:
    t.join()
    
```

1번에서는 첫번째 스레드가 시작되고, 그 스레드가 종료 될때까지 모든 스레드는 사용할 수 없다.
반면에 2번에서는 모든 스레드가 동시에 시작되고, 그 후에 모든 서브 스레드를 join()으로 동기화하기 때문에 메인스레드만 남는듯

### 클래스 상속을 통해서 스레드를 만드는 방법

In [14]:
import threading
from time import sleep, ctime

In [15]:
print("시작시간: ", ctime())

class Hands(threading.Thread):
    def run(self):
        sleep(3)
        print(ctime())
        print("손을 흔들흔들")
        
class Legs(threading.Thread):
    def run(self):
        sleep(2)
        print(ctime())
        print("다리를 흔들흔들")
    
class Head(threading.Thread):
    def run(self):
        sleep(1)
        print(ctime())
        print("머리를 쉐킷쉐킷")
        
hand = Hands()
leg = Legs()
head = Head()

hand.start()
leg.start()
head.start()

시작시간:  Sat Aug 29 23:15:26 2020


In [16]:
# 상속을 통해서 만드는 방법

import threading
from time import sleep, ctime

class Test(threading.Thread):
    
    def __init__(self, func): # 생성자를 오버라이드 할 경우에는 반드시 부모 클래스의 생성자 호출
        threading.Thread.__init__(self)
        self.func = func
        
    # Thread를 상속받은 클래스는 반드시 run 메서드를 오버라이드 해야한다.
    def run(self):
        self.func()
        
def hands():
    sleep(3)
    print(ctime())
    return print("손을 흔든다.")

def legs():
    sleep(2)
    print(ctime())
    return print("발을 흔든다.")

def head():
    sleep(1)
    print(ctime())
    return print("머리를 휘젓는다.")

In [17]:
a = Test(hands)
b = Test(legs)
c = Test(head)

threads = [a, b, c]

for t in threads:
    t.start()

상속을 통한 스레드도 결국 스레드 객체를 생성해서, 실행시키는 것은 동일하다.

단, threading.Thread를 상속 받은 클래스에 run 메서드를 오버라이드 해서 스레드 시작시에 실행할 코드를 작성하는 것이다.

# 동기화

스레드를 사용하면 한 프로세스 내에 점유한 스레드끼리 메모리가 공유되기 때문에 데이터 교환 측면에서는 유리하지만, 모든 전역 변수들이 같은 객체를 참조하기 때문에 발생하는 문제가 발생할 수 있다. 이를 방지하기 위한 수단이 동기화!

## threading 모듈의 대표적인 동기화 수단

* Lock: 한번에 하나의 스레드만 사용할 수 있는 자원의 접근 구간에 락을 설치하면, 락을 획득한 1개의 스레드만 실행 되고, 나머지는 기다리게 된다.

#### 락, Lock

칸이 하나밖에 없는 화장실이다.

누군가(스레드)가 락을 획득하면, 나머지(다른 스레드)는 락이 풀릴 때까지 잠긴다.

Lock을 사용하기 위해서는 스레드 외부에서 Lock 인스턴스를 생성한다. 
그리고 한정된 자원을(화장실)을 엑세스 하려는 시점에서 `Lock.acquire()` 메서드를 호출한다.

만약 락을 획득하면 자신의 상태를 변경하며 즉시 리턴한다.(화장실에 들어가면 바로 자물쇠를 채움)<br>
하지만 다른 스레드가 선점하여 Lock이 된 상태라면, 기다린다.

리소스 사용을 마치면(화장실 사용이 끝나면) `Lock.release()`를 호출해서 자물쇠를 해제한다. 그러면 또 다음 스레드가 이런 과정을 반복한다.

간단한 자원 경쟁의 예시를 보자.

두개의 스레드가 하나의 값을 읽은 후 거기에 1을 더한다.

하지만 아무런 보호가 없기 때문에 변경한 후 값을 보면 1이 아닌 2씩 더해지는 것을 확인할 수 있다.

In [18]:
names = ['철수', '민수']

In [22]:
# 동기화를 하지 않은 경우, 동시에 들어간다 ㅋㅋ;

def toilet(name):
    print(f"{name}이(가) 화장실에 들어갑니다.\n")
    sleep(1)
    print(f"{name}이(가) 화장실에서 나옵니다.\n")

철수 = threading.Thread(target = toilet, args=(names[0],))
민수 = threading.Thread(target = toilet, args=(names[1],))
    
# threads = [threading.Thread(target=toilet, args=(n,)) for n in names]

철수.start()
민수.start()

철수이(가) 화장실에 들어갑니다.

민수이(가) 화장실에 들어갑니다.

철수이(가) 화장실에서 나옵니다.

민수이(가) 화장실에서 나옵니다.



In [21]:
# 동기화를 해본다. 
# 이 경우엔 철수 스레드가 끝난 후에 민수 스레드가 시작되는 것을 확인 할 수 있다.

lock = threading.Lock() # lock 객체를 만들어주고

def toilet(name):
    lock.acquire()
    print(f"{name}이(가) 화장실에 들어갑니다.\n")
    sleep(3)
    print(f"{name}이(가) 화장실에서 나옵니다.\n")
    lock.release()

threads = [threading.Thread(target=toilet, args=(n,)) for n in names]

for t in threads:
    t.start()
    
# with 구문 활용하면 acquire, release가 필요 없다.

철수이(가) 화장실에 들어갑니다.

철수이(가) 화장실에서 나옵니다.

민수이(가) 화장실에 들어갑니다.

민수이(가) 화장실에서 나옵니다.

